<a href="https://colab.research.google.com/github/Shreeshambav/DeepLearning_training/blob/main/Deeplearning_LSTM_Backpropagation_Bidirectional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [3]:
## Solving Fake news classifier using Backpropagation
import pandas as pd
# df=pd.read_csv('train.csv',engine='python',error_bad_lines=False)
df=pd.read_csv('train.csv')

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.shape

(20800, 5)

In [6]:
df=df.dropna()

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
## Get the Independent features
X=df.drop('label',axis=1)

In [9]:
## Get the dependent feature
y=df['label']

In [10]:
# Check whether the dataset is balanced or not - Dataset is not balanced
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.12.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
## vocabulary size
voc_size=5000

In [15]:
messages=X.copy()

In [16]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [17]:
# Stopwords are used to remove unnecessary words
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
# Since we have removed NAN value- Let us reindex
messages.reset_index(inplace=True)

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [23]:
# My input feature
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
# We need to apply one hot encoding
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4257, 1759, 4465, 2615, 247, 892, 4958, 4829, 2085, 3245],
 [1134, 1481, 602, 4194, 644, 57, 2512],
 [2101, 2224, 3408, 2562],
 [3801, 4263, 2953, 4435, 4356, 1331],
 [2243, 644, 1959, 3304, 2948, 2069, 644, 2300, 2198, 3989],
 [1210,
  1738,
  4423,
  2026,
  2368,
  394,
  4555,
  621,
  416,
  1744,
  4049,
  1407,
  3834,
  4292,
  2512],
 [1721, 1195, 2991, 3293, 2703, 3845, 2771, 1502, 489, 42, 4703],
 [357, 3207, 2415, 4662, 154, 4801, 394, 2753, 489, 42, 4703],
 [2841, 3399, 3704, 2837, 60, 3872, 2199, 664, 394, 2277],
 [1475, 3726, 3960, 3940, 3517, 4436, 4578, 4036],
 [3869, 1815, 4590, 849, 4349, 4924, 3337, 1759, 3712, 4523, 3278],
 [4435, 4008, 247, 3872, 394, 154],
 [1094, 2294, 275, 4125, 2147, 1591, 1846, 2685, 2723],
 [1583, 104, 4489, 1283, 165, 3730, 3297, 489, 42, 4703],
 [685, 4835, 4463, 2072, 3142, 489, 42, 4703],
 [4558, 199, 2603, 4551, 4123, 2654, 930, 3641, 979, 3982],
 [861, 3827, 1481],
 [1059, 4524, 2217, 3065, 394, 3476, 632, 2512],
 [4045, 2599, 602, 1

In [25]:
## Embedding Representation - Words into vectors
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 4829, 2085, 3245],
       [   0,    0,    0, ...,  644,   57, 2512],
       [   0,    0,    0, ..., 2224, 3408, 2562],
       ...,
       [   0,    0,    0, ...,  489,   42, 4703],
       [   0,    0,    0, ..., 1680, 4601,   59],
       [   0,    0,    0, ..., 4249,  809, 1048]], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
# Converted embedded array into array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 26s 44ms/step - loss: 0.2610 - accuracy: 0.8853 - val_loss: 0.1954 - val_accuracy: 0.9221
Epoch 2/10
383/383 [==============================] - 5s 14ms/step - loss: 0.1224 - accuracy: 0.9523 - val_loss: 0.1946 - val_accuracy: 0.9191
Epoch 3/10
383/383 [==============================] - 4s 11ms/step - loss: 0.0825 - accuracy: 0.9707 - val_loss: 0.2264 - val_accuracy: 0.9145
Epoch 4/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0528 - accuracy: 0.9808 - val_loss: 0.2677 - val_accuracy: 0.9205
Epoch 5/10
383/383 [==============================] - 5s 12ms/step - loss: 0.0321 - accuracy: 0.9889 - val_loss: 0.3215 - val_accuracy: 0.9138
Epoch 6/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0197 - accuracy: 0.9929 - val_loss: 0.4476 - val_accuracy: 0.9107
Epoch 7/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0159 - accuracy: 0.9946 - val_loss: 0.3809 - val_accuracy: 0.9148

In [30]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [31]:
# Keeping threshold as 0.5
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3105,  314],
       [ 195, 2421]])

In [33]:
print(accuracy_score(y_test,y_pred))

0.9156586578293289
